In [1]:
import PyPDF2
import os
import openai
# from dotenv import load_dotenv
from langchain.chat_models import AzureChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import AzureSearch
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader
from langchain.text_splitter import TokenTextSplitter
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate
from azure.identity import DefaultAzureCredential
from azure.search.documents.indexes.models import HnswVectorSearchAlgorithmConfiguration

In [2]:
llm = AzureChatOpenAI(
    openai_api_key = "7439c61b0745446dbbb029521f498265",
    openai_api_base = "https://ai-dev-openai-fts.openai.azure.com/",
    openai_api_version = "2023-07-01-preview",
    temperature=0,    
    deployment_name="gpt35")

In [3]:
embeddings = OpenAIEmbeddings(
    openai_api_key = "7439c61b0745446dbbb029521f498265",
    openai_api_base = "https://ai-dev-openai-fts.openai.azure.com/",
    openai_api_type = "azure",
    openai_api_version = "2023-07-01-preview",
    deployment="text-embedding-ada-002", 
    chunk_size=1
    )

In [4]:
acs = AzureSearch(azure_search_endpoint="https://ai-dev-ss-fts.search.windows.net",
                 azure_search_key="ICyDmM0S3sWaLpYwNGPfUOz7jblRyOHnE6p7uDHPRRAzSeAaLBFM",
                 index_name="client-docs",
                 embedding_function= embeddings.embed_query
                 )

In [18]:
# def parse_response(response):
#     print(response['result'])
#     print('\n\nSources:')
#     for source_name in response["source_documents"]:
#         print(source_name.metadata['source'])
#         break

In [8]:
import re

def parse_response(response):
    print(response['result'])
    print('\nSources:')
    for source_name in response["source_documents"]:
        #print("Test :",source_name)
        source_name = re.search(r'[^\\/:*?"<>|\r\n]+$', source_name.metadata['source']).group()
        print(source_name)
        break

In [9]:
from langchain.chains import RetrievalQA

retriever = acs.as_retriever(include_metadata=True, metadata_key = 'source')

chain  = RetrievalQA.from_chain_type(llm=llm, chain_type= "stuff", retriever=retriever, return_source_documents=True)

In [19]:
query = "How insight.com reporting will help your business ?"

response = chain(query)

parse_response(response)

Insight.com reporting can help your business by providing clear visibility into your IT spend, allowing you to analyze, understand, and reconcile your technology expenditures. The reports are recognized universally for their accuracy, and because Insight offers global procurement, they can meet your global reporting needs. They offer multiple sharable formats, all print-ready, and a full suite of standard reports that will meet most, if not all, of your data analysis needs. When you need more, you can customize quickly. This analysis-ready data can make all the difference when negotiating with vendors or projecting budgets. Additionally, online reporting data is available when you need it, and customized reports can be generated at a moment's notice. Finally, reconciliation is dramatically streamlined, allowing you to easily reconcile expenditures by location, cost centers, general ledger codes, or other custom data.


Sources:
3 - Reporting (1).txt


In [10]:
query = "What is the headcount for Manager Director segmant ?"

response = chain(query)

parse_response(response)

The headcount for Manager Director segment is 1.

Sources:
TSEGermanymemodraft.txt


In [22]:
query = "What is comparable uncontrolled price method ?"

response = chain(query)

parse_response(response)

The comparable uncontrolled price (CUP) method is a transfer pricing method that compares the price charged for property or services transferred in a controlled transaction to the price charged for property or services transferred in a comparable uncontrolled transaction in comparable circumstances. For the purposes of the CUP method, an uncontrolled transaction is considered to be comparable to a controlled transaction if reasonably accurate adjustments can be made to eliminate the effects of any material differences between the controlled and uncontrolled transactions. The CUP method is a particularly reliable method where an independent enterprise sells or buys the same product or service as that sold between the two related parties concerned.


Sources:
C:\Users\achauha2\Downloads\test-txt\TSEGermanymemodraft.txt
